## Some useful function while exploring WatNet

1. Check raster resolution 

In [ ]:
# check tif resolution 
import rasterio

# Path to your GeoTIFF file
file_path = output_path

# Open the GeoTIFF file
with rasterio.open(file_path) as src:
    # Get the transform (affine matrix) of the GeoTIFF file
    transform = src.transform
    
    # Extract the pixel size in meters from the transform
    pixel_width = transform.a
    pixel_height = -transform.e  # Negative becuse the y-coordinate increases downward
    
    # Print the pixel size in meters
    print(f"Pixel size (width, height): {pixel_width:.2f} meters, {pixel_height:.2f} meters")

2. Add color interpretation for raster bands

In [ ]:
# add color interpretation for raster bands 
from osgeo import gdal
# Open the raster dataset
dataset = gdal.Open(sen_raster_path, gdal.GA_Update)

# Set color interpretation for a specific band (e.g., band 1)
band = dataset.GetRasterBand(1)
band.SetColorInterpretation(gdal.GCI_RedBand)  # Set as red band

band = dataset.GetRasterBand(2)
band.SetColorInterpretation(gdal.GCI_GreenBand)  # Set as green band

band = dataset.GetRasterBand(3)
band.SetColorInterpretation(gdal.GCI_BlueBand)  # Set as blue band
# Close the dataset to save changes
dataset = None

3. Count bands, print band information, check whether it has NIR band

In [ ]:
# this is not from watnet, wanted to check the bands 
def count_bands(tif_path):
    try:
        dataset = gdal.Open(tif_path)
        if dataset is None:
            print("Unable to open", tif_path)
            return None
        else:
            num_bands = dataset.RasterCount
            print("Number of bands:", num_bands)
            return num_bands
    except Exception as e:
        print("Error:", e)

def print_band_info(tif_path):
    try:
        dataset = gdal.Open(tif_path)
        if dataset is None:
            print("Unable to open", tif_path)
            return None
        else:
            num_bands = dataset.RasterCount
            print("Number of bands:", num_bands)
            for band_num in range(1, num_bands + 1):
                band = dataset.GetRasterBand(band_num)
                print("Band", band_num, "information:")
                print("  Type:", gdal.GetDataTypeName(band.DataType))
                print("  Minimum value:", band.GetMinimum())
                print("  Maximum value:", band.GetMaximum())
                print("  NoData value:", band.GetNoDataValue())
                print("  Color interpretation:", gdal.GetColorInterpretationName(band.GetColorInterpretation()))
                print("  Scale factor:", band.GetScale())
                print("  Offset:", band.GetOffset())
                print("  Unit type:", band.GetUnitType())
                print("  Metadata:", band.GetMetadata())
                print("  Description:", band.GetDescription())
                print("")

    except Exception as e:
        print("Error:", e)

def check_nir_band(raster_path):
    # Open the raster dataset
    dataset = gdal.Open(raster_path)

    if dataset is None:
        print("Error: Unable to open raster dataset.")
        return False

    # Get the number of bands
    num_bands = dataset.RasterCount

    # Check if the NIR band is present
    for i in range(1, num_bands + 1):
        band = dataset.GetRasterBand(i)
        metadata = band.GetMetadata()

        # Check the band description or color interpretation
        band_desc = metadata.get('description', '')
        color_interp = band.GetColorInterpretation()

        if 'NIR' in band_desc or color_interp == gdal.GCI_GrayIndex:
            print(f"NIR band found in band {i}.")
            return True

    print("NIR band not found in the raster.")
    return False

4. composite 

In [ ]:
# composite raster 
import arcpy
arcpy.management.CompositeBands(
    in_rasters="2021-07-10-00_00_2021-07-10-23_59_Sentinel-2_L2A_B04_(Raw).tiff;2021-07-10-00_00_2021-07-10-23_59_Sentinel-2_L2A_B03_(Raw).tiff;2021-07-10-00_00_2021-07-10-23_59_Sentinel-2_L2A_B02_(Raw).tiff;2021-07-10-00_00_2021-07-10-23_59_Sentinel-2_L2A_B08_(Raw).tiff;2021-07-10-00_00_2021-07-10-23_59_Sentinel-2_L2A_B11_(Raw).tiff;2021-07-10-00_00_2021-07-10-23_59_Sentinel-2_L2A_B12_(Raw).tiff",
    out_raster=r"H:\My Drive\Projects\Beavers\Beaver_EarthEngine\Sentinel2_6Bands_WatNet\Browser_images\compositeBand_20210710_polygonA.tif"
)